In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('data/data.txt', delimiter='\t')

In [3]:
# Create dateimte from year, month and day
data['date'] = pd.to_datetime(data['billing_year'].astype(str) + '-' + data['billing_month'].astype(str) + '-' + data['billing_day'].astype(str))
data.set_index('date', inplace=True)

In [4]:
data

,report_month,billing_year,billing_month,billing_cycle,billing_day,billing_amount
date,,,,,,
2023-02-26,3/1/2023,2023,2,26,26,2.105541e+08
2023-02-27,3/1/2023,2023,2,27,27,6.444154e+07
2023-02-28,3/1/2023,2023,2,28,28,2.535149e+08
2023-02-28,3/1/2023,2023,2,29,28,5.535407e+07
2023-02-28,3/1/2023,2023,2,30,28,5.350947e+07
...,...,...,...,...,...,...
2024-07-21,7/1/2024,2024,7,21,21,NaN
2024-07-22,7/1/2024,2024,7,22,22,NaN
2024-07-23,7/1/2024,2024,7,23,23,NaN


In [5]:
# Feature engineering
data['day_of_month'] = data.index.day
data['day_of_week'] = data.index.dayofweek
data['month'] = data.index.month
data['year'] = data.index.year


In [10]:
data

,report_month,billing_year,billing_month,billing_cycle,billing_day,billing_amount,day_of_month,day_of_week,month,year
date,,,,,,,,,,
2023-02-26,3/1/2023,2023,2,26,26,2.105541e+08,26,6,2,2023
2023-02-27,3/1/2023,2023,2,27,27,6.444154e+07,27,0,2,2023
2023-02-28,3/1/2023,2023,2,28,28,2.535149e+08,28,1,2,2023
2023-02-28,3/1/2023,2023,2,29,28,5.535407e+07,28,1,2,2023
2023-02-28,3/1/2023,2023,2,30,28,5.350947e+07,28,1,2,2023
...,...,...,...,...,...,...,...,...,...,...
2024-07-21,7/1/2024,2024,7,21,21,NaN,21,6,7,2024
2024-07-22,7/1/2024,2024,7,22,22,NaN,22,0,7,2024
2024-07-23,7/1/2024,2024,7,23,23,NaN,23,1,7,2024


In [12]:
# Calculate rolling averages based on billing cycle
data['rolling_avg_3m'] = data.groupby('billing_cycle')['billing_amount'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
data['rolling_avg_6m'] = data.groupby('billing_cycle')['billing_amount'].transform(lambda x: x.rolling(window=6, min_periods=1).mean())

In [14]:
# Lag features
data['lag_1'] = data.groupby('billing_cycle')['billing_amount'].shift(1, axis=0)
data['lag_1'] = data.groupby('billing_cycle')['lag_1'].apply(lambda x: x.ffill())

TypeError: incompatible index of inserted column with frame index

In [13]:



data['lag_7'] = data.groupby('billing_cycle')['billing_amount'].shift(7, axis=0)
data['lag_7'] = data.groupby('billing_cycle')['lag_7'].apply(lambda x: x.bfill())

TypeError: incompatible index of inserted column with frame index

In [ ]:
data

In [ ]:
future_data = data.loc['2024-07-09':'2024-07-25']

data.dropna(inplace=True)

In [ ]:
# Split data into train and test sets by report month
train = data.loc[data['report_month'] < data['report_month'].max()]
test = data.loc[data['report_month'] == data['report_month'].max()]

In [15]:
data

,report_month,billing_year,billing_month,billing_cycle,billing_day,billing_amount,day_of_month,day_of_week,month,year,rolling_avg_3m,rolling_avg_6m,lag_1
date,,,,,,,,,,,,,
2023-02-26,3/1/2023,2023,2,26,26,2.105541e+08,26,6,2,2023,2.105541e+08,2.105541e+08,NaN
2023-02-27,3/1/2023,2023,2,27,27,6.444154e+07,27,0,2,2023,6.444154e+07,6.444154e+07,NaN
2023-02-28,3/1/2023,2023,2,28,28,2.535149e+08,28,1,2,2023,2.535149e+08,2.535149e+08,NaN
2023-02-28,3/1/2023,2023,2,29,28,5.535407e+07,28,1,2,2023,5.535407e+07,5.535407e+07,NaN
2023-02-28,3/1/2023,2023,2,30,28,5.350947e+07,28,1,2,2023,5.350947e+07,5.350947e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-21,7/1/2024,2024,7,21,21,NaN,21,6,7,2024,2.227068e+08,2.215748e+08,2.242816e+08
2024-07-22,7/1/2024,2024,7,22,22,NaN,22,0,7,2024,2.193223e+08,2.185852e+08,2.206858e+08
2024-07-23,7/1/2024,2024,7,23,23,NaN,23,1,7,2024,3.237422e+08,3.231236e+08,3.242328e+08


In [16]:
# Features and target
features = ['billing_cycle','day_of_month','day_of_week','month','year','lag_1','lag_7','rolling_avg_3m','rolling_avg_6m']
target = 'billing_amount'

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

NameError: name 'train' is not defined

In [ ]:
# Preprocessing for numeric and categorial features
numeric_features = ['billing_cycle','day_of_month','day_of_week','month','year','lag_1','lag_7','rolling_avg_3m','rolling_avg_6m']

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features)])
        #('cat', OneHotEncoder(), categorical_features)])

In [ ]:
# Gradient Boosting pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', GradientBoostingRegressor(n_estimators=100
                                                                   ,learning_rate=0.1
                                                                   ,max_depth=3
                                                                   ,random_state=42))])

In [ ]:
# Model training
pipeline.fit(X_train, y_train)
predictions_gb = pipeline.predict(X_test)

# Evaluation
mae_gb = mean_absolute_error(y_test, predictions_gb)
r2 = r2_score(y_test, predictions_gb)
print(f'Gradient Boosting MAE: {mae_gb}')
print(f'Gradient Boosting R^2 Score: {r2}')

In [ ]:
# Plot actual vs predicted values
plt.figure(figsize=(12,6))
plt.plot(y_test.index, y_test, label='Actual')
plt.plot(y_test.index, predictions_gb, label='Predicted', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Billing Amount')
plt.title('Actual vs Predicted Billing Amount')
plt.legend()
plt.show()

In [ ]:
# Create DataFrame with actual vs predicted values
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions_gb}, index=y_test.index)
results_df['Error %'] = ((results_df['Predicted'] - results_df['Actual']) / results_df['Actual']) * 100
results_df['Actual'] = results_df['Actual'].apply(lambda x: f'{x:,.2f}')
results_df['Predicted'] = results_df['Predicted'].apply(lambda x: f'{x:,.2f}')
results_df['Error %'] = results_df['Error %'].apply(lambda x: f'{x:.2f}%')

results_df

# Predict future billing amounts
future_predictions = pipeline.predict(future_data[features])

# Create DataFrame for future predictions
future_results_df = pd.DataFrame({'Predicted': future_predictions}, index=future_data.index)
future_results_df['Predicted'] = future_results_df['Predicted'].apply(lambda x: f'{x:,.2f}')

future_results_df